In [1]:
import os
import pandas as pd
import nltk
import gensim
import numpy as np
from gensim import corpora, models, similarities

c:\users\devan\appdata\local\programs\python\python36\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
df=pd.read_json('./cnn.json')

In [3]:
x = df['title'].values.tolist()
y = df['content'].values.tolist()

In [4]:
corpus = x+y

In [5]:
tok_corp = [nltk.word_tokenize(sent) for sent in corpus]

In [6]:
model = gensim.models.Word2Vec(tok_corp,min_count=1,size=32)

In [7]:
model.most_similar('I')

c:\users\devan\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('his', 0.9995924234390259),
 ('and', 0.9995760917663574),
 ('of', 0.9995478391647339),
 ('by', 0.9995389580726624),
 (',', 0.9995273351669312),
 ('to', 0.9995264410972595),
 ('that', 0.999498188495636),
 ('are', 0.9994771480560303),
 ('a', 0.9994707107543945),
 ('is', 0.999467134475708)]

In [8]:
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector

c:\users\devan\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [9]:
def visualize(model, output_path):
    meta_file = "w2x_metadata.tsv"
    placeholder = np.zeros((len(model.wv.index2word), 32))

    with open(os.path.join(output_path,meta_file), 'wb') as file_metadata:
        for i, word in enumerate(model.wv.index2word):
            placeholder[i] = model[word]
            # temporary solution for https://github.com/tensorflow/tensorflow/issues/9094
            if word == '':
                print("Emply Line, should replecaed by any thing else, or will cause a bug of tensorboard")
                file_metadata.write("{0}".format('<Empty Line>').encode('utf-8') + b'\n')
            else:
                file_metadata.write("{0}".format(word).encode('utf-8') + b'\n')

    # define the model without training
    sess = tf.InteractiveSession()

    embedding = tf.Variable(placeholder, trainable = False, name = 'w2x_metadata')
    tf.global_variables_initializer().run()

    saver = tf.train.Saver()
    writer = tf.summary.FileWriter(output_path, sess.graph)

    # adding into projector
    config = projector.ProjectorConfig()
    embed = config.embeddings.add()
    embed.tensor_name = 'w2x_metadata'
    embed.metadata_path = meta_file

    # Specify the width and height of a single thumbnail.
    projector.visualize_embeddings(writer, config)
    saver.save(sess, os.path.join(output_path,'w2x_metadata.ckpt'))
    print('Run `tensorboard --logdir={0}` to run visualize result on tensorboard'.format(output_path))

In [11]:
# visualize(model,"./")

c:\users\devan\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


Run `tensorboard --logdir=./` to run visualize result on tensorboard


In [12]:
model.save('model_cnn_edition')